# Unlabeled Data Analysis

create twitter hashtag reader and reddit thread reader

pick sample hashtags and threads

## Twitter

In [131]:
import nest_asyncio
import twint
nest_asyncio.apply()

In [132]:
c = twint.Config()
c.Search = "#cyberpunk"
c.Min_likes = 5

In [133]:
twint.run.Search(c)

CRITICAL:root:twint.run:Twint:Feed:noDataExpecting value: line 1 column 1 (char 0)
sleeping for 1.0 secs
CRITICAL:root:twint.run:Twint:Feed:noDataExpecting value: line 1 column 1 (char 0)
sleeping for 8.0 secs


KeyboardInterrupt: 

## Reddit

In [1]:
from praw import Reddit
from config import reddit_api

In [112]:
def get_comments(url):
    reddit = Reddit(client_id=reddit_api['client_id'],
                    client_secret=reddit_api['client_secret'],
                    user_agent=reddit_api['user_agent'])

    submissionId = url[url.find('comments'):].split('/')[1]
    submission = reddit.submission(submissionId)
    submission.comments.replace_more(limit=None)
    comments = []
    for comment in submission.comments:
        comments.append(comment.body)
        
    return comments

In [113]:
X = get_comments('https://www.reddit.com/r/Games/comments/ls33hs/activision_warns_a_standard_500gb_ps4_may_no/')

load best processing and model pickles

## Processing Data

In [19]:
from keras.models import load_model
import pandas as pd
from pickle import load
import preprocessing

In [14]:
def process_data(X):
    X_pre = list(map(preprocessing.remove_markdown, X))
    X_pre = list(map(preprocessing.remove_punctuation, X_pre))
    X_pre = list(map(preprocessing.tokenize, X_pre))
    X_pre = list(map(preprocessing.lemmatize, X_pre))
    X_join = [' '.join(x) for x in X_pre]
    
    return X_join

In [115]:
model = load_model('final_model/model.h5')
model.load_weights('final_model/model_weights.h5')
vectorizer = load(open('final_model/vectorizer.pk', 'rb'))
    
def get_reddit_predictions(url):
    comments = get_comments(url)
    X = process_data(comments)
    
    X_processed = vectorizer.transform(process_data(X)).todense()
    y_pred = model.predict_classes(X_processed)
    df = pd.DataFrame(zip(X, y_pred.flatten()), columns=['comment', 'positive'])
    
    return df

sample reddit thread ideas:
- https://www.reddit.com/r/nintendo/comments/lm6obv/project_triangle_strategy_announcement_trailer/
- https://www.reddit.com/r/nintendo/comments/lru33p/the_animal_crossing_new_horizons_free_update_is/
- https://www.reddit.com/r/nintendo/comments/lqlrqg/tony_hawks_pro_skater_1_2_coming_to_nintendo/
- https://www.reddit.com/r/Games/comments/ls6qlh/bravery_default_ii_review_thread/

In [127]:
df = get_reddit_predictions('https://www.reddit.com/r/Games/comments/lm6ns2/project_triangle_strategy_announcement_trailer/')

C:\Users\Andrew\anaconda3\envs\capstone\lib\site-packages\tensorflow\python\keras\engine\sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [128]:
df['positive'].value_counts(normalize=True)

1    0.909091
0    0.090909
Name: positive, dtype: float64

In [129]:
df['comment'][df[df['positive']==True].index].tolist()[:5]

['ff tactic in the octopath engine yes please',
 'this is basically octopath traveler for final fantasy tactic right with the basic plot premise kind of similar to three house if so thats amazing i think what we know so far definitely fix a lot of the issue a lot of people had with octopath',
 'politics with tactical strategy rpg sign me up',
 'this look interesting like octopath im sure it will have solid combat visuals and musicthe thing that ha me most excited though is what look like a more focused story and not 8 separate plot the character story and writing in octopath i thought were really lacking so if they improved it this time around i can see this game being fantastic i really like the aspect of choice it advertising a well',
 'this is so exciting there a serious dearth of great srpgs these day they come but few and far between this one look incredibly interesting and im hoping it can help to fill that voidmoreover this game reminds me a lot of tactic ogre let u cling togeth

In [130]:
df['comment'][df[df['positive']==False].index].tolist()[:5]

['so if they go with the same strategy a octopath traveler the final name for this will be triangle strategy',
 'terrible title that seems like it will be changed so thats goodfeels a lot like a spiritual successor to final fantasy tactic so im very excited for itthe branching path gameplay seems like something that is always promised in game that end up being lackluster in execution with the choice not really changing much in the actual story but ill be interested to see how much actually changesall in all im excited to try out the demo and see how this turn out kind of disappointing it coming out next year but oh well',
 'another entry in the terrible title franchise should be fun though didnt vibe too much with octopath but this look pretty good im always in favor of more srpgs',
 'i feel like i know benedict voice actor but i cant put my finger on it anyone have any idea',
 'it sound promising so far ill keep an eye on iti hope the choice will actually matter in which case we shoul

run data through processing and model

output class percentages and numbers

output unlabeled topic analysis

CREATE FRONTEND!!!